- SOLID 원칙
    - S: 단일 책임 원칙 (Single Responsibility Principle)
    - O: 개방-폐쇄 원칙 (Open/Closed Principle)
    - L: 리스코프 치환 원칙 (Liskov Substitution Principle)
    - I: 인터페이스 분리 원칙 (Interface Segregation Principle)
    - D: 의존 역전 원칙 (Dependency Inversion Principle)

- 이 장의 목표
    - 소프트웨어 디자인에서의 SOLID 원칙을 익힌다
    - 단일 책임 원칙을 따르는 컴포넌트를 디자인한다
    - 개방/폐쇄 원칙을 통해 유지보수성을 뛰어나게 한다
    - 리스코프 치환 원칙을 준수하여 객체지향 디자인에서 적절한 클래스 계층을 설계한다
    - 인터페이스 분리와 의존성 역전을 활용해 설계하기

# 단일 책임 원칙 Single Responsibility Principle
- 소프트웨어 컴포넌트 (일반적으로 클래스) 는 단 하나의 책임을 져야 한다
    - 단 하나의 구체적인 일을 담당한다
    - 변경이 필요한 이유도 단 하나만 있어야 한다 - 해당 도메인의 문제가 변경되는 경우만 클래스가 업데이트 되어야 한다
- 신 god 객체
    - 필요한 일 이상의 것을 하거나 너무 많은 것을 알고 있는 객체
    - 서로 다른 행동을 그룹화하였으므로 유지보수가 어렵다

## 너무 많은 책임을 가진 클래스

In [ ]:
class SystemMonitor:
    def load_activity(self):
        """소스에서 처리할 이벤트를 가져오기"""
        print("Loading activity...")

    def identify_events(self):
        """가져온 데이터를 파싱하여 도메인 객체 이벤트로 변환"""
        print("Identifying events...")

    def stream_event(self):
        """파싱한 이벤트를 외부 에이전트로 전송"""
        print("Streaming event...")

- 이 클래스의 문제점
    - 독립적인 동작을 하는 메서드를 하나의 인터페이스에 정의
    - 유지보수가 어려움 -> 클래스가 경직되고 융통성이 없으며 오류가 발생하기 쉬움
- 대안: 추상화된 loader 메서드
    - 데이터 소스에 연결, 데이터 로드. 예상된 형식으로 파싱
    - 데이터 구조를 바꾸는 등의 이유로 이 중의 어떤 것이라도 수정해야 한다면 SystemMonitor 클래스를 변경해야 한다.
    - 하지만 그러한 디자인이 정당한가? 아님

## 책임 분산
- AlertSystem
    - has a ActivityWatcher.load(): Events
    - has a SystemMonitor.itentify(Event)
    - has a Output.stream(List\[Event\])
- 이렇게 구현하면 변경 사항이 로컬에만 적용되고 영향이 미미하여 유지보수가 쉽다
- ActivityWatcher, SystemMonitor, Output은 인터페이스로 구현하여 상속받음

# 개방/폐쇄 원칙 (OCP)
- 개방/폐쇄 원칙 Open/Close Principle
    - 모듈이 개방되어 있으면서도 폐쇄되어야 한다는 원칙
    - 유지보수가 쉽도록 로직을 캡슐화하여 확장에는 개방, 수정에는 폐쇄되어야 한다


## 개방/폐쇄 원칙을 따르지 않을 경우 유지보수의 어려움

In [1]:
from dataclasses import dataclass


@dataclass
class Event:
    raw_data: dict


class UnknownEvent(Event):
    """A type of event that cannot be identified from its data."""


class LoginEvent(Event):
    """A event representing a user that has just entered the system."""


class LogoutEvent(Event):
    """An event representing a user that has just left the system."""


class SystemMonitor:
    """Identify events that occurred in the system

    >>> l1 = SystemMonitor({"before": {"session": 0}, "after": {"session": 1}})
    >>> l1.identify_event().__class__.__name__
    'LoginEvent'

    >>> l2 = SystemMonitor({"before": {"session": 1}, "after": {"session": 0}})
    >>> l2.identify_event().__class__.__name__
    'LogoutEvent'

    >>> l3 = SystemMonitor({"before": {"session": 1}, "after": {"session": 1}})
    >>> l3.identify_event().__class__.__name__
    'UnknownEvent'

    """

    def __init__(self, event_data):
        self.event_data = event_data

    def identify_event(self):
        if (
            self.event_data["before"]["session"] == 0
            and self.event_data["after"]["session"] == 1
        ):
            return LoginEvent(self.event_data)
        elif (
            self.event_data["before"]["session"] == 1
            and self.event_data["after"]["session"] == 0
        ):
            return LogoutEvent(self.event_data)

        return UnknownEvent(self.event_data)

In [2]:
l1 = SystemMonitor({"before": {"session": 0}, "after": {"session": 1}})
l1.identify_event().__class__.__name__

'LoginEvent'

In [3]:
l2 = SystemMonitor({"before": {"session": 1}, "after": {"session": 0}})
l2.identify_event().__class__.__name__

'LogoutEvent'

In [4]:
l3 = SystemMonitor({"before": {"session": 1}, "after": {"session": 1}})
l3.identify_event().__class__.__name__

'UnknownEvent'

- 문제점
    - 이벤트 유형을 결정하는 로직이 identify_event에 집중되어 있음
        - 이벤트가 늘어날수록 메서드가 커짐
        - 새로운 이벤트를 추가할때마다 메서드를 수정해야 함
        - elif 문이 가독성에 최악
- 메서드를 변경하지 않고 새로운 유형의 이벤트를 추가하는 것이 폐쇄 원칙
- 새로운 이벤트를 추가할 때 이미 존재하는 코드를 변경하지 않고 코드를 확장하여 새로운 이벤트를 지원하는 것이 개방 원칙

## 확장성을 가진 이벤트 시스템으로 리팩토링

In [5]:
from dataclasses import dataclass


@dataclass
class Event:
    raw_data: dict

    @staticmethod
    def meets_condition(event_data: dict) -> bool:
        return False


class UnknownEvent(Event):
    """A type of event that cannot be identified from its data"""


class LoginEvent(Event):
    @staticmethod
    def meets_condition(event_data: dict):
        return (
            event_data["before"]["session"] == 0
            and event_data["after"]["session"] == 1
        )


class LogoutEvent(Event):
    @staticmethod
    def meets_condition(event_data: dict):
        return (
            event_data["before"]["session"] == 1
            and event_data["after"]["session"] == 0
        )


class SystemMonitor:
    """Identify events that occurred in the system

    >>> l1 = SystemMonitor({"before": {"session": 0}, "after": {"session": 1}})
    >>> l1.identify_event().__class__.__name__
    'LoginEvent'

    >>> l2 = SystemMonitor({"before": {"session": 1}, "after": {"session": 0}})
    >>> l2.identify_event().__class__.__name__
    'LogoutEvent'

    >>> l3 = SystemMonitor({"before": {"session": 1}, "after": {"session": 1}})
    >>> l3.identify_event().__class__.__name__
    'UnknownEvent'

    """

    def __init__(self, event_data):
        self.event_data = event_data

    def identify_event(self):
        for event_cls in Event.__subclasses__():
            try:
                if event_cls.meets_condition(self.event_data):
                    return event_cls(self.event_data)
            except KeyError:
                continue
        return UnknownEvent(self.event_data)

## 이벤트 시스템 확장

In [6]:
class TransactionEvent(Event):
    """Represents a transaction that has just occurred on the system."""

    @staticmethod
    def meets_condition(event_data: dict):
        return event_data["after"].get("transaction") is not None

In [7]:
l4 = SystemMonitor({"after": {"transaction": "Tx001"}})
l4.identify_event().__class__.__name__

'TransactionEvent'

- 새 이벤트를 추가했지만 SystemMonitor는 수정하지 않았음
- 폐쇄 & 개방 원칙을 따르고 있음

## OCP 최종 정리
- 다형성의 효과적인 사용과 밀접하게 관련되어 있음
    - 다형성을 따르는 형태의 계약을 만들고 모델을 쉽게 확장할 수 있는 일반적인 구조로 디자인하는 것
    - "객체 지향 프로그래밍에서 다형성(Polymorphism)은 객체 지향의 주요 개념 중 하나로, 같은 이름의 메서드나 연산자가 다른 클래스에 대해 다른 동작을 하도록 하는 것을 말합니다."
- 보호하려는 추상화 (e.g. 새로운 이벤트 유형) 에 대해서 적절한 폐쇄를 해야한다.
    - 일부 추상화에서는 충돌이 발생할 수 있기 때문에 모든 프로그램에서 이 원칙을 적용할 수 있는 것은 아니다.

# 리스코프 치환 원칙 (LSP)
- Liskov substitution principle
    - 클라이언트가 특별한 주의를 기울이지 않고도 부모클래스를 대신하여 하위 클래스를 그대로 사용할 수 있어야 한다
    - 클라이언트는 어떤 하위 타입을 사용해도 정상적으로 동작해야 한다는 뜻
- LISKOV 01: 만약 S가 T의 하위 타입이라면 프로그램을 변경하지 않고 T 타입의 객체를 S 타입의 객체로 치환 가능해야 한다.


## 도구를 사용해 LSP 문제 검사하기

### mypy로 잘못된 메서드 서명 검사
- 파생 클래스가 부모 클래스에서 정의한 파라미터와 다른 타입을 사용하면 LSP를 위반 -> mypy가 알려줌
- 상호 교환이 가능하다는 것은
    - 기본 클래스의 메시지를 모두 구현했고 
    - 선언되지 않은 다른 public 메서드를 구현하지 않았고
    - 메서드의 서명을 변경하지 않았다

### pylint로 호환되지 않는 서명 검사
- 메서드의 서명 자체가 완전히 다르면 탐지하기 어렵다
- mypy로 오류를 잡고 pylint로 통찰을 얻을 수 있다

### 애매한 LSP 위반 사례
- 계약이 수정되면 자동으로 감지하기 어렵다
    - 계약은 계층 어디서나 항상 유지되어야 한다
    - 부모 클래스의 계약을 하위 클래스가 따라야 한다.
        - 하위 클래스가 부모 클래스에 정의된 것 보다 사전 조건을 엄격하게 만들면 안 된다
        - 하위 클래스는 부모 클래스에 정의된 것 보다 약한 사후조건을 만들면 안 된다

In [8]:
from collections.abc import Mapping


class Event:
    def __init__(self, raw_data):
        self.raw_data = raw_data

    @staticmethod
    def meets_condition(event_data: dict) -> bool:
        return False

    @staticmethod
    def validate_precondition(event_data: dict):
        """Precondition of the contract of this interface.

        Validate that the ``event_data`` parameter is properly formed.
        """
        if not isinstance(event_data, Mapping):
            raise ValueError(f"{event_data!r} is not a dict")
        for moment in ("before", "after"):
            if moment not in event_data:
                raise ValueError(f"{moment} not in {event_data}")
            if not isinstance(event_data[moment], Mapping):
                raise ValueError(f"event_data[{moment!r}] is not a dict")


class UnknownEvent(Event):
    """A type of event that cannot be identified from its data"""


class LoginEvent(Event):
    @staticmethod
    def meets_condition(event_data: dict) -> bool:
        return (
            event_data["before"].get("session") == 0
            and event_data["after"].get("session") == 1
        )


class LogoutEvent(Event):
    @staticmethod
    def meets_condition(event_data: dict) -> bool:
        return (
            event_data["before"].get("session") == 1
            and event_data["after"].get("session") == 0
        )


class TransactionEvent(Event):
    """Represents a transaction that has just occurred on the system."""

    @staticmethod
    def meets_condition(event_data: dict) -> bool:
        return event_data["after"].get("transaction") is not None


class SystemMonitor:
    """Identify events that occurred in the system

    >>> l1 = SystemMonitor({"before": {"session": 0}, "after": {"session": 1}})
    >>> l1.identify_event().__class__.__name__
    'LoginEvent'

    >>> l2 = SystemMonitor({"before": {"session": 1}, "after": {"session": 0}})
    >>> l2.identify_event().__class__.__name__
    'LogoutEvent'

    >>> l3 = SystemMonitor({"before": {"session": 1}, "after": {"session": 1}})
    >>> l3.identify_event().__class__.__name__
    'UnknownEvent'

    >>> l4 = SystemMonitor({"before": {}, "after": {"transaction": "Tx001"}})
    >>> l4.identify_event().__class__.__name__
    'TransactionEvent'

    """

    def __init__(self, event_data):
        self.event_data = event_data

    def identify_event(self):
        Event.validate_precondition(self.event_data)
        event_cls = next(
            (
                event_cls
                for event_cls in Event.__subclasses__()
                if event_cls.meets_condition(self.event_data)
            ),
            UnknownEvent,
        )
        return event_cls(self.event_data)

### LSP 최종 정리

# 인터페이스 분리 원칙
- Interface Segregation Principle
    - 인터페이스: 객체가 노출하는 메서드의 집합
- 덕 타이핑의 원리: 어떤 새가 오리처럼 걷고 오리처럼 꽥꽥 소리를 낸다면 오리여야만 한다
    - 모든 객체가 자신이 가지고 있는 메서드와 자신이 할 수 있는 일에 의해서 표현된다
    - 클래스의 다른 속성에 관계 없이 객체의 본질을 정의하는 것은 궁극적으로 메서드의 형태이다.
- 추상 기본 클래스 abstract base class
    - 파생 클래스가 반드시 구현해야 하는 것을 명시적으로 가리키기 위한 도구
    - 위에서 Event를 추상 기본 클래스로 지정하면 Event 클래스는 인터페이스로 동작해서 각 파생 클래스가 meats_conditions 메서드를 구현하도록 강제한다.
    - 방법: @abstractmethod 데코레이터 사용
- 가상 서브 클래스 virtual subclass
    - ABC의 register()를 사용해서 기존 기본 클래스에 파생 클래스를 추가
    - 오리처럼 행동하는 새로운 오리 등록하기
- ISP의 추상적인 의미
    - 여러 메서드를 가진 인터페이스가 있다면 매우 정확하고 구체적인 구분에 따라 더 적은 수의 메서드를 가진 여러개의 인터페이스로 분할하는 것이 좋다.
    - 재사용성을 높이기 위해 가능한 작은 단위로 인터페이스를 분리한다면 인터페이스를 구현하려는 각 클래스가 매우 명확한 동작과 책임을 지니기 때문에 응집력이 높아진다.

## 너무 많은 일을 하는 인터페이스
- xml 과 json을 파싱하는 클래스
    - 나누는게 좋다

## 인터페이스는 작을수록 좋다

In [2]:
from abc import ABCMeta, abstractmethod


class XMLEventParser(metaclass=ABCMeta):
    @abstractmethod
    def from_xml(xml_data: str):
        """Parse an event from a source in XML representation."""


class JSONEventParser(metaclass=ABCMeta):
    @abstractmethod
    def from_json(json_data: str):
        """Parse an event from a source in JSON format."""


class EventParser(XMLEventParser, JSONEventParser):
    """An event parser that can create an event from source data either
    in XML or JSON format.
    """

    def from_xml(xml_data):
        pass

    def from_json(json_data: str):
        pass

In [3]:
EventParser() # ????

In [4]:
EventParser().from_xml()

- SRP와 유사하지만 ISP는 인터페이스에 대해 이야기하고 있다 = 행동의 추상화

## 인터페이스는 얼마나 작아야 할까?
- 추상 클래스든 기본 클래스든 "인터페이스를 정의" 하는건 같음
    - 응집력의 관점에서 가능한 단 한가지 일을 수행하는 작은 인터페이스여야 함
    - 그러나 딱 한가지 메서드를 의미하는 것은 아님
        - e.g. 컨텍스트 매니저

# 의존성 역전
- Dependency Inverse Principle
    - 코드가 깨지거나 손상되는 취약점으로부터 보호해주는 디자인 원칙
    - 코드가 세부사항이나 구체적인 구현에 적응하도록 하지 않고, API 같은 것에 적응하도록 하는 것
    - 추상화를 통해 세부사항에 의존하지 않도록 해야 하지만, 반대로 세부 사항 (구체적인 구현) 은 추상화에 의존해야 한다. <-?
- 예시
    - A와 B가 상호 교류
    - A는 B 인스턴스를 사용하지만 B 모듈을 직접 관리하지는 않는다.
    - 코드가 B에 크게 의존하면 B 코드가 변경되었을 때 원래의 코드가 쉽게 깨지게 된다
    - 어떻게 해야할까? B가 A에 적응하게 하자

## 강한 의존성을 가진 예
- EventStreamer 클래스 (목표지) 에 전송하는 이벤트 전송 클래스 Syslog
- 저수준의 내용에 따라 고수준의 클래스가 변경되어야 하므로 좋은 디자인이 아니다.
    - Syslog가 변경되면 EventStreamer를 수정해야 한다.

## 의존성을 거꾸로
- 대안: EventStreamer를 인터페이스와 대화하도록 하기
    - EventStreamer가 구체적인 구현을 하지 않고, 인터페이스와 대화하도록 한다.
    - DataTargetClient를 부모 클래스로 두고 파생 클래스로 Syslog를 만들어서 이것을 EventStreamer에게 인스턴스로 넘겨준다.
    - 그러면 나중에 send 함수가 바뀌었을 때 DataTargetClient의 다른 파생 클래스를 EventStreamer에게 넘겨주기만 하면 된다.
- 의존성을 동적으로 제공하기 때문에 의존성 주입이라고도 한다.
- 상속을 꼭 구현해야 하는것은 아니지만, 가독성과 (is a 관계) 실수를 줄이기 위해 권장된다.

## 의존성 주입 (Dependency injection)

In [13]:
from __future__ import annotations

import json

from abc import ABCMeta, abstractmethod


class Event:
    def __init__(self, content: dict) -> None:
        self._content = content

    def serialise(self):
        return json.dumps(self._content)


class DataTargetClient(metaclass=ABCMeta):
    @abstractmethod
    def send(self, content: bytes):
        """Send raw content to a particular target."""


class Syslog(DataTargetClient):
    def send(self, content: bytes):
        return f"[{self.__class__.__name__}] sent {len(content)} bytes"


class EventStreamer:
    def __init__(self, target: DataTargetClient):
        self.target = target

    def stream(self, events: list[Event]) -> None:
        for event in events:
            self.target.send(event.serialise())

- 만약 복잡한 초기화 과정을 가졌거나 초기화 인자가 많다면, 종속성 그래프를 만들고 관련 라이브러리가 생성을 담당하도록 하는 것도 좋은 방법

In [5]:
from __future__ import annotations

import json

from abc import ABCMeta, abstractmethod

import pinject


class Event:
    def __init__(self, content: dict) -> None:
        self._content = content

    def serialise(self):
        return json.dumps(self._content)


class DataTargetClient(metaclass=ABCMeta):
    @abstractmethod
    def send(self, content: bytes):
        """Send raw content to a particular target."""


class Syslog(DataTargetClient):
    def send(self, content: bytes):
        return f"[{self.__class__.__name__}] sent {len(content)} bytes"


class EventStreamer:
    def __init__(self, target: DataTargetClient):
        self.target = target

    def stream(self, events: list[Event]) -> None:
        for event in events:
            self.target.send(event.serialise())


class _EventStreamerBindingSpec(pinject.BindingSpec):
    def provide_target(self):
        return Syslog()


object_graph = pinject.new_object_graph(
    binding_specs=[_EventStreamerBindingSpec()]
)

ModuleNotFoundError: No module named '_gdbm'

- 팩토리 객체와 유사
- 의존성을 주입한다고 해서 원래 디자인에서 얻은 유연성을 잃은 것은 아니다.

# 요약
- 소프트웨어 빌드는 어려운 작업이다.
    - 로직은 복잡하고 런타임에서의 동작은 예측활 수 없을 수 있고 요구사항은 지속적으로 변하고 환경도 끊임없이 변한다.
    - 다양한 기술 패러다임을 사용한 디자인으로 구성이 바뀔 수 있으며 요구사항이 변경되면 접근방식이 바뀌어야 할 수도 있다.
- SOLID 원칙에 충실하면 수년 후에도 융통성 있게 변화에 적응할 수 있게 구현할 수 있다.